In [1]:
pip install google-cloud-secret-manager

In [22]:
pip install --upgrade google-cloud-aiplatform


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.39.0
    Uninstalling google-cloud-aiplatform-1.39.0:
      Successfully uninstalled google-cloud-aiplatform-1.39.0


In [2]:
!pip install google-cloud-aiplatform google-cloud-storage pandas sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [8]:
pip install google-cloud-secret-manager

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from google.cloud.secretmanager_v1 import SecretManagerServiceClient

import os
from google.cloud import storage
from google.cloud import aiplatform
from google.cloud import bigquery

import json

In [2]:

# Carga las variables de configuración desde Secret Manager
client = SecretManagerServiceClient()  # Use the correctly imported class
secret_name = "iris-secret"
secret_version = "latest"
response = client.access_secret_version(request={"name": f"projects/{'molten-unison-414815'}/secrets/{secret_name}/versions/{secret_version}"})  # Add project ID

# ... rest of your code to handle the secret

secret_payload = response.payload.data.decode("UTF-8")
config = json.loads(secret_payload)

PROJECT_ID = config["PROJECT_ID"]
BUCKET_NAME = config["BUCKET_NAME"]
REGION = config["REGION"]


In [3]:
df = pd.read_csv("iris.csv")


In [4]:
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y = df['species']


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [18]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [22]:
from google.cloud import aiplatform as vertexai
import time
dataset = vertexai.TabularDataset.create(
    display_name=f"iris-dataset-{int(time.time())}",
    gcs_source=["gs://renzo_bucket_test/data/iris.csv"],
)


INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/279275268617/locations/us-central1/datasets/2712702993421041664/operations/8405968635767554048
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/279275268617/locations/us-central1/datasets/2712702993421041664
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/279275268617/locations/us-central1/datasets/2712702993421041664')


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from google.cloud import bigquery
from google.cloud import secretmanager
from google.cloud import aiplatform as vertexai  # Importar Vertex AI


# ... el resto de tu código


In [30]:
from google.cloud import aiplatform as vertexai

In [34]:
model = train.AutoMLClassificationModel(
    project_id=PROJECT_ID,
    location=REGION,
    dataset=dataset,
    model_name=f"iris-model-{int(time.time())}",
    training_fraction=0.8,
    validation_fraction=0.2,
)


NameError: name 'AutoMLTabularTrainingJob' is not defined

In [ ]:
model.wait_for_completion()


In [ ]:
metrics = model.evaluate(
    split="test",
    metrics=["accuracy", "f1_score", "precision", "recall"]
)


In [ ]:
metrics = model.evaluate(
    split="test",
    metrics=["accuracy", "f1_score", "precision", "recall"]
)


In [ ]:
# Crea un pipeline
pipeline = pipelines.Pipeline(
    name=f"iris-pipeline-{int(time.time())}",
    project_id=PROJECT_ID,
    location=REGION
)

# Agrega las tareas al pipeline
pipeline.add_task(
    name="load_data",
    task_factory=load_data_task_factory,
    inputs=dict(
        bucket_name=BUCKET_NAME,
        filename="iris.csv"
    )
)

pipeline.add_task(
    name="train_model",
    task_factory=train_model_task_factory,
    inputs=dict(
        dataset_name=task_inputs["load_data"].outputs["dataset_name"]
    )
)

pipeline.add_task(
    name="generate_predictions",
    task_factory=generate_predictions_task_factory,
    inputs=dict(
        model_name=task_inputs["train_model"].outputs["model_name"],
        X_test=task_inputs["load_data"].outputs["X_test"]
    )
)

pipeline.add_task(
    name="save_predictions",
    task_factory=save_predictions_task_factory,
    inputs=dict(
        predictions=task_inputs["generate_predictions"].outputs["predictions"],
        project_id=PROJECT_ID,
        dataset_id="iris_dataset",
        table_id="predictions"
    )
)

# Compila y ejecuta el pipeline
pipeline.compile()
pipeline.run()


In [ ]:
def save_predictions(predictions, project_id, dataset_id, table_id):

  # Crea un cliente de BigQuery
  client = bigquery.Client(project=project_id)

  # Crea un dataset si no existe
  dataset = client.create_dataset(dataset_id, exists_ok=True)

  # Crea una tabla si no existe
